In [3]:
import fitz
import camelot
import warnings , math, collections
import pickle
from collections import Counter
import re

warnings.filterwarnings("ignore", category=UserWarning) 

#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"


pickle_path_text = r"\output\pkl\textual_data.pkl"
pickle_path_tab = r"\output\pkl\tabular_data.pkl"
samco_path = r"\files\SamcoFactSheet2024.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"
dry_run_path = r"\output\DryRun.pdf"

no_image_pkl = r"\output\pkl\sam\nonimg_data.pkl"

In [4]:
with open(path + no_image_pkl , 'rb') as file:
    document_data = pickle.load(file)  

In [5]:
len(document_data) #ignore first and last page

2

In [7]:
fonts = set()

for page in document_data['blocks_data']:
    sortedPage = sorted(page['blocks'], key= lambda k: (round(k['bbox'][1]), round(k['bbox'][0]))) #sort t to b, l to r
    firstBlock = sortedPage[0] #get first block of sorted data
    
    if 'lines' in firstBlock:
        for line in firstBlock['lines']:
            for span in line['spans']:
                text = span['text'].lower()
                font = span['font']
                fonts.add(font)
                
                #regex condition
                cond1 = re.findall('^samco|^tata|fund$', text)
                if cond1:
                    print(text)
                

samco active momentum fund
samco active momentum fund
samco flexi cap fund
samco flexi cap fund
samco dynamic asset allocation fund
samco dynamic asset allocation fund
samco multi cap fund
samco multi cap fund
samco special opportunities fund
samco special opportunities fund
samco elss tax saver fund
samco elss tax saver fund
samco overnight fund
samco dynamic asset allocation fund 
samco overnight fund
samco elss tax saver fund 


In [7]:
"""Extract and sort lines for a two-column layout, returning all data associated with each line without duplicates.
    Args:data (list): The raw extracted data from Fitz.
        split_x_coord (float): The x-coordinate dividing the left and right columns.
    Returns:list: Sorted list of all unique line data in reading order (top-to-bottom, left-to-right).
    """
def sort_data_blocks(data, split_x_coord):
    # Separate lines into left and right columns
    left_column = []
    right_column = []

    for item in data:
        for line in item.get('lines', []):
            bbox = line.get('bbox', None)
            spans = line.get('spans', [])
            if bbox and spans:
                # Classify line based on its x-coordinate
                block = {'bbox': bbox, 'spans': spans, 'line': line}
                if bbox[0] < split_x_coord:  # Left column
                    left_column.append(block)
                else:  # Right column
                    right_column.append(block)

    # Sort lines within each column
    left_column.sort(key=lambda x: (x['bbox'][1], x['bbox'][0]))
    right_column.sort(key=lambda x: (x['bbox'][1], x['bbox'][0]))

    return left_column, right_column

In [58]:
left, right = sort_data_blocks(document_data['blocks_data'][17]['blocks'], 173)

In [10]:
def remove_keys(data, keys_to_remove):
    """Recursively remove specific keys from nested dictionaries or lists.
    Args:data (dict | list): The input data (nested structure of dictionaries and lists).
        keys_to_remove (set): Keys to be removed from the dictionaries.
    Returns:dict | list: Data with specified keys removed."""
    if isinstance(data, list):
        # Process each element in the list
        return [remove_keys(item, keys_to_remove) for item in data]
    elif isinstance(data, dict):
        # Process each key-value pair in the dictionary
        return {key: remove_keys(value, keys_to_remove) for key, value in data.items() if key not in keys_to_remove}
    else:
        # Return data as is if it's neither a dict nor a list
        return data

In [59]:
for line in left:
    del line['spans']
    
keys_to_remove = {'bbox','wmode', 'ascender', 'descender',}

In [3]:
cleaned_data = remove_keys(left, keys_to_remove)

NameError: name 'remove_keys' is not defined

In [ ]:
cleaned_data

In [2]:
"""Create a nested dictionary structure using a matrix-based approach.
    Args:
        data (list): List of text blocks, each containing font size, text, and bbox.
        title_font (float): Font size for titles.
        subheader_font (float): Font size for subheaders.
        content_max_font (float): Maximum font size for content.
    Returns:
        dict: A nested dictionary structure with titles, subheaders, and content.
    """
    
def create_matrix_structure(data, title_font, subheader_font, content_max_font):

    # Step 1: Extract all unique coordinates
    coordinates = []
    fonts = set()
    for block in data:
        for span in block['line']['spans']:
            bbox = block['line'].get('bbox', (0, 0, 0, 0))
            coordinates.append((bbox[0], bbox[1]))  # x, y from bbox
            fonts.add(span['size'])

    coordinates = sorted(set(coordinates), key=lambda c: (c[1], c[0]))  # Sort by y, then x
    fonts = sorted(fonts)

    # Step 2: Initialize the matrix
    coord_to_index = {coord: idx for idx, coord in enumerate(coordinates)}
    font_to_index = {font: idx for idx, font in enumerate(fonts)}
    matrix = np.zeros((len(coordinates), len(fonts)), dtype=object)

    # Step 3: Fill the matrix
    for block in data:
        for span in block['line']['spans']:
            bbox = block['line'].get('bbox', (0, 0, 0, 0))
            coord = (bbox[0], bbox[1])
            font = span['size']
            if coord in coord_to_index and font in font_to_index:
                row = coord_to_index[coord]
                col = font_to_index[font]
                if matrix[row, col] == 0:
                    matrix[row, col] = []
                matrix[row, col].append(span['text'])

    # Step 4: Generate the nested dictionary
    nested_dict = {}
    current_title = None
    current_subheader = None

    for row_idx, coord in enumerate(coordinates):
        for col_idx, font in enumerate(fonts):
            if matrix[row_idx, col_idx] != 0:
                text = " ".join(matrix[row_idx, col_idx])

                if font == title_font:
                    current_title = text
                    nested_dict[current_title] = {}
                elif font == subheader_font and current_title:
                    current_subheader = text
                    nested_dict[current_title][current_subheader] = []
                elif font <= content_max_font and current_subheader:
                    nested_dict[current_title][current_subheader].append(text)

    return nested_dict


# Example usage
title_font = 24.0  # Example title font size
subheader_font = 9.0  # Example subheader font size
content_max_font = 8.0  # Example maximum content font size

# Call the function with your data
result = create_matrix_structure(left, title_font, subheader_font, content_max_font)


NameError: name 'cleaned_data' is not defined

In [ ]:
# Print the result
import pprint
pprint.pprint(result)